# Test for the Modified Number Operator
## Construct the Hamiltonians for 1d lattice

In [1]:
import sys
sys.path.append("../")
import fh_comm as fhc
import numpy as np

In [2]:
v = -1
u = 1
translatt = fhc.SubLattice(np.array([[2]]))
h0 = fhc.SumOp([fhc.HoppingOp(( 0,), ( 1,), s, v) for s in [0, 1]])
h1 = fhc.SumOp([fhc.HoppingOp((-1,), ( 0,), s, v) for s in [0, 1]])
h2 = fhc.SumOp([fhc.ProductOp([fhc.ModifiedNumOp((x,), s, 1) for s in [0, 1]], u) for x in [0, 1]])
h2_num = [[fhc.NumberOp((x,),s,1.) for s in [0,1]] for x in [0,1]]

In [3]:
# evaluate bound for 3 Hamiltonian terms
comm_bound_terms = fhc.commutator_bound_strang(3)
for cbt in comm_bound_terms:
    print(cbt)

1/24 * [H_0, [H_1, H_0]]
1/12 * [H_1, [H_1, H_0]]
1/12 * [H_2, [H_1, H_0]]
1/24 * [H_0, [H_2, H_0]]
1/12 * [H_1, [H_2, H_0]]
1/12 * [H_2, [H_2, H_0]]
1/24 * [H_1, [H_2, H_1]]
1/12 * [H_2, [H_2, H_1]]


Notice: Because we just modified H_2, than we just need to compute the term with H_2,
s.t. [H_2, [H_1, H_0]], [H_0, [H_2, H_0]], [H_1, [H_2, H_0]], [H_2, [H_2, H_0]], [H_1, [H_2, H_1]], [H_2, [H_2, H_1]]

In [4]:
# results of the first level commutator
result10 = fhc.commutator(h1,h0)
result20 = fhc.commutator(h2,h0)
result21 = fhc.commutator(h2,h1)

In [5]:
# results of the 6 terms with H_2
result = []
result.append(fhc.commutator(h2,result10))
result.append(fhc.commutator(h0,result20))
result.append(fhc.commutator(h1,result20))
result.append(fhc.commutator(h2,result20))
result.append(fhc.commutator(h1,result21))
result.append(fhc.commutator(h2,result21))
for i in range(len(result)):
    result[i] = fhc.simplify(result[i])

In [8]:
# for convinience, we construct a function to compute the modified number operator by hand
def theretical_h2_form(a: fhc.HamiltonianOp)->fhc.HamiltonianOp:
    return fhc.SumOp([fhc.commutator(fhc.ProductOp([h2_num[0][0], h2_num[0][1]],u),a), fhc.commutator(fhc.ProductOp([h2_num[1][0], h2_num[1][1]],u),a), 0.5*u*fhc.commutator(a, fhc.SumOp([Op for sublist in h2_num for Op in sublist]))])

In [10]:
# we compute the results by hand, and then compare the theoretical results with those above
the_result = []
the_result20 = theretical_h2_form(h0)
the_result21 = theretical_h2_form(h1)
the_result.append(theretical_h2_form(result10))
the_result.append(fhc.commutator(h0, the_result20))
the_result.append(fhc.commutator(h1, the_result20))
the_result.append(theretical_h2_form(the_result20))
the_result.append(fhc.commutator(h1, the_result21))
the_result.append(theretical_h2_form(the_result21))
for i in range(len(the_result)):
    the_result[i] = fhc.simplify(the_result[i])

In [18]:
# to check whether the 6 terms are the same, the outpus should be all 'True'
for i in range(len(result)):
    print(np.array_equal(result[i].as_field_operator().as_matrix((3,)).toarray(), the_result[i].as_field_operator().as_matrix((3,)).toarray()))

True
True
True
True
True
True


Here, we could try to construct the higher order suzuki, and try to print out the err bound